In [2]:
pip install wandb

  Obtaining dependency information for wandb from https://files.pythonhosted.org/packages/28/3b/f1485df03e33a390b833081693e56be9e62fef097a82c26ef615605f768d/wandb-0.16.2-py3-none-any.whl.metadata
  Obtaining dependency information for GitPython!=3.1.29,>=1.0.0 from https://files.pythonhosted.org/packages/45/c6/a637a7a11d4619957cb95ca195168759a4502991b1b91c13d3203ffc3748/GitPython-3.1.41-py3-none-any.whl.metadata
  Obtaining dependency information for sentry-sdk>=1.0.0 from https://files.pythonhosted.org/packages/f6/1a/d40a3fbc24f365a3891dce9967cd5f6869840572dcd66981f38b92cc3357/sentry_sdk-1.39.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for setproctitle from https://files.pythonhosted.org/packages/ff/5d/77edf4c29c8d6728b49d3f0abb22159bb9c0c4ddebd721c09486b34985c8/setproctitle-1.3.3-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for gitdb<5,>=4.0.1 from https://files.pythonhosted.org/packages/fd/5b/8f0c4a5bb9fd491c277c21eff7ccae71b4

In [6]:
import wandb
import random
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
import numpy.ma as ma
import glob
from astropy.table import Table
import astropy.units as u
from scipy import interpolate
from astropy.io import fits, ascii

In [7]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [8]:
wandb.init(
    project = 'YSOs Identifying Model'
)

In [9]:
model = tf.keras.Sequential()
model.add(layers.Conv1D(128, (3), activation='relu', padding='same',input_shape=(8575, 1)))
model.add(layers.MaxPooling1D((2)))
model.add(layers.Conv1D(256, (3), activation='relu'))
model.add(layers.MaxPooling1D((2)))
model.add(layers.Conv1D(256, (3), activation='relu'))
model.add(layers.MaxPooling1D((2)))
model.add(layers.Conv1D(256, (3), activation='relu'))
model.add(layers.MaxPooling1D((2)))
model.add(layers.Conv1D(256, (3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(2))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 8575, 128)         512       
                                                                 
 max_pooling1d (MaxPooling1  (None, 4287, 128)         0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 4285, 256)         98560     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 2142, 256)         0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 2140, 256)         196864    
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 1070, 256)        

In [14]:
class DataGenerator(tf.keras.utils.Sequence):
    """
    Custom data generator class for Digits dataset
    """
    def __init__(self, path,shuffle=True):

        
        self.paths=path
        self.shuffle = shuffle
        
        np.random.seed(123)

    def __len__(self):
        return len(self.paths)

    def on_epoch_end(self):
        if self.shuffle:
            a=np.random.uniform(size=len(self.paths))
            b=np.argsort(a)
            self.paths=self.paths[b]
            
    def trans(self,x):
        if x == 0: return [0]
        bit = []
        while x:
            bit.append(x % 2)
            x >>= 1
        return bit#[::-1]


    def __getitem__(self, index):
        """
        Returns a batch of data
        """
        
        j=index
    
        hdul = fits.open(self.paths[j])
        ff=hdul[1].data
        flagg=hdul[3].data
        hdul.close()
        if ff.ndim==1:
            f=ff
            flag=flagg
        else:
            f=ff[0]
            flag=flagg[0]
        hdr=fits.getheader(self.paths[j])
        nf=len(f)
        crval=np.double(hdr['CRVAL1'])
        cdelt=np.double(hdr['CDELT1'])
        w=10**(np.arange(nf)*cdelt+crval)/1e4
        x1=np.zeros((len(flag),15))
        for i in range(len(flag)):
            z=self.trans(flag[i])
            x1[i,:len(z)]=z
        k=np.where((x1[:,0]==1) | (x1[:,12]==1)| (x1[:,8]==1) | (np.isfinite(f)==False))[0]
        f[k]=np.nanmedian(f)
        f=f/np.nanmedian(f)
        labels=np.array([np.round(np.random.random())])
    
        f=f.reshape((1,len(f),1))
        
        return f,labels
    
specs = np.array(['apStar-daily-apo25m-2M04263451+3733098.fits','apStar-daily-apo25m-2M04264097+3734222.fits','apStar-daily-apo25m-2M04265369+3717527.fits','apStar-daily-apo25m-2M04270905+3713589.fits','apStar-daily-apo25m-2M04274252+3742282.fits'])
        
dev_loader = DataGenerator(specs,shuffle=True)
x=dev_loader.__getitem__(1)
print(x)

(array([[[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]]]), array([1.]))


In [21]:
def train(model):
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])   
    
    wandb_callbacks = [
        WandbMetricsLogger(log_freq=1),
        #WandbClfEvalCallback(
        #                validloader=dev_loader,data_table_columns=[],pred_table_columns=[]
        #                        ),
        WandbModelCheckpoint(filepath='ebs',save_weights_only=False,restore_best_weights=True,
                                    verbose=1,save_best_only= True,monitor='val_accuracy',mode='max'),
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode = 'max',patience=5)
    ]
    
    model.fit(dev_loader, epochs=10,callbacks = [wandb_callbacks])
    

In [23]:
train(model)

Epoch 1/10
5/5 [==============================] - ETA: 0s - loss: 0.6996 - accuracy: 0.2000    WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


wandb: Adding directory to artifact (./ebs)... Done. 0.0s


5/5 [==============================] - 16s 2s/step - loss: 0.6996 - accuracy: 0.2000
Epoch 2/10
5/5 [==============================] - ETA: 0s - loss: 0.6944 - accuracy: 0.6000WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


wandb: Adding directory to artifact (./ebs)... Done. 0.0s


5/5 [==============================] - 7s 2s/step - loss: 0.6944 - accuracy: 0.6000
Epoch 3/10
5/5 [==============================] - ETA: 0s - loss: 0.7022 - accuracy: 0.2000    WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


...

5/5 [==============================] - 9s 2s/step - loss: 0.6945 - accuracy: 0.2000
Epoch 9/10
5/5 [==============================] - ETA: 0s - loss: 0.6942 - accuracy: 0.2000WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


wandb: Adding directory to artifact (./ebs)... Done. 0.0s


5/5 [==============================] - 8s 2s/step - loss: 0.6942 - accuracy: 0.2000
Epoch 10/10
5/5 [==============================] - ETA: 0s - loss: 0.6929 - accuracy: 0.6000WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


wandb: Adding directory to artifact (./ebs)... Done. 0.0s


5/5 [==============================] - 8s 2s/step - loss: 0.6929 - accuracy: 0.6000


In [24]:
wandb.finish()

batch/accuracy,▁▃▁▅▅▆▅▇█▁▆█▆▅▅▅▅▃▁▅█▃▁▂▆▅▁█▆▅▆▅▇▁▁▃▁▃▅▅
batch/batch_step,▁▁▁▁▁▁▂▂▃▃▃▄▄▅▅▅▆▆▇▇▇█▁▂▂▂▃▃▄▄▄▅▅▆▆▆▇▇██
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,▂█▃▃▅▃▂▂▁▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
epoch/accuracy,▅▆▆█▆▅▆▆▆▆▆▃▃▆▃█▆█▁▃▃▆
epoch/epoch,▁▁▁▂▃▃▄▅▆▆▇█▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/accuracy,0.6
batch/batch_step,49
batch/learning_rate,0.001
